## 自编码器实战

In [1]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from    tensorflow.keras import Sequential, layers
from    PIL import Image
from    matplotlib import pyplot as plt

In [2]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [3]:
def save_images(imgs, name):
    new_im = Image.new('L', (280, 280))

    index = 0
    for i in range(0, 280, 28):
        for j in range(0, 280, 28):
            im = imgs[index]
            im = Image.fromarray(im, mode='L')
            new_im.paste(im, (i, j))
            index += 1

    new_im.save(name)

In [4]:
h_dim = 20
batchsz = 512
lr = 1e-3

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train.astype(np.float32) / 255., x_test.astype(np.float32) / 255.
# we do not need label
train_db = tf.data.Dataset.from_tensor_slices(x_train)
train_db = train_db.shuffle(batchsz * 5).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices(x_test)
test_db = test_db.batch(batchsz)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [6]:
class AE(keras.Model):

    def __init__(self):
        super(AE, self).__init__()

        # Encoders
        self.encoder = Sequential([
            layers.Dense(256, activation=tf.nn.relu),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(h_dim)
        ])

        # Decoders
        self.decoder = Sequential([
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(256, activation=tf.nn.relu),
            layers.Dense(784)
        ])


    def call(self, inputs, training=None):
        # [b, 784] => [b, 10]
        h = self.encoder(inputs)
        # [b, 10] => [b, 784]
        x_hat = self.decoder(h)

        return x_hat


In [7]:
model = AE()
model.build(input_shape=(None, 784))
model.summary()

Model: "ae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  236436    
_________________________________________________________________
sequential_1 (Sequential)    multiple                  237200    
Total params: 473,636
Trainable params: 473,636
Non-trainable params: 0
_________________________________________________________________


In [8]:
optimizer = tf.optimizers.Adam(lr=lr)

for epoch in range(100):

    for step, x in enumerate(train_db):

        #[b, 28, 28] => [b, 784]
        x = tf.reshape(x, [-1, 784])

        with tf.GradientTape() as tape:
            x_rec_logits = model(x)

            rec_loss = tf.losses.binary_crossentropy(x, x_rec_logits, from_logits=True)
            rec_loss = tf.reduce_mean(rec_loss)

        grads = tape.gradient(rec_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))


        if step % 100 ==0:
            print(epoch, step, float(rec_loss))


        # evaluation
        x = next(iter(test_db))
        logits = model(tf.reshape(x, [-1, 784]))
        x_hat = tf.sigmoid(logits)
        # [b, 784] => [b, 28, 28]
        x_hat = tf.reshape(x_hat, [-1, 28, 28])

        # [b, 28, 28] => [2b, 28, 28]
        x_concat = tf.concat([x, x_hat], axis=0)
        x_concat = x_hat
        x_concat = x_concat.numpy() * 255.
        x_concat = x_concat.astype(np.uint8)
        save_images(x_concat, 'ae_images/rec_epoch_%d.png'%epoch)

0 0 0.6926943063735962
0 100 0.32507020235061646
1 0 0.3100969195365906
1 100 0.3056202828884125
2 0 0.2954757511615753
2 100 0.3020476698875427
3 0 0.2970770299434662
3 100 0.2959977090358734
4 0 0.29245081543922424
4 100 0.2939869463443756
5 0 0.28266698122024536
5 100 0.2879142761230469
6 0 0.28452861309051514
6 100 0.2896577715873718
7 0 0.2819896638393402
7 100 0.2801777720451355
8 0 0.28772565722465515
8 100 0.2901213765144348
9 0 0.27792859077453613
9 100 0.2786189913749695
10 0 0.28806501626968384
10 100 0.2799556851387024
11 0 0.2749875485897064
11 100 0.2814224064350128
12 0 0.2764913737773895
12 100 0.2829265594482422
13 0 0.2741786241531372
13 100 0.27818989753723145
14 0 0.2745482623577118
14 100 0.2845202088356018
15 0 0.2798682749271393
15 100 0.2762281894683838
16 0 0.26946574449539185
16 100 0.2783936560153961
17 0 0.27365803718566895
17 100 0.2754528522491455
18 0 0.27423095703125
18 100 0.28272610902786255
19 0 0.27456432580947876
19 100 0.2764202356338501
20 0 0.279

## 变分自动编码器实战

In [9]:
z_dim = 10

class VAE(keras.Model):

    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = layers.Dense(128)
        self.fc2 = layers.Dense(z_dim) # get mean prediction
        self.fc3 = layers.Dense(z_dim)

        # Decoder
        self.fc4 = layers.Dense(128)
        self.fc5 = layers.Dense(784)

    def encoder(self, x):

        h = tf.nn.relu(self.fc1(x))
        # get mean
        mu = self.fc2(h)
        # get variance
        log_var = self.fc3(h)

        return mu, log_var

    def decoder(self, z):

        out = tf.nn.relu(self.fc4(z))
        out = self.fc5(out)

        return out

    def reparameterize(self, mu, log_var):

        eps = tf.random.normal(log_var.shape)

        std = tf.exp(log_var*0.5)

        z = mu + std * eps
        return z

    def call(self, inputs, training=None):

        # [b, 784] => [b, z_dim], [b, z_dim]
        mu, log_var = self.encoder(inputs)
        # reparameterization trick
        z = self.reparameterize(mu, log_var)

        x_hat = self.decoder(z)

        return x_hat, mu, log_var

In [10]:
model = VAE()
model.build(input_shape=(4, 784))
optimizer = tf.optimizers.Adam(lr)

for epoch in range(1000):

    for step, x in enumerate(train_db):

        x = tf.reshape(x, [-1, 784])

        with tf.GradientTape() as tape:
            x_rec_logits, mu, log_var = model(x)

            rec_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=x_rec_logits)
            rec_loss = tf.reduce_sum(rec_loss) / x.shape[0]

            # compute kl divergence (mu, var) ~ N (0, 1)
            # https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
            kl_div = -0.5 * (log_var + 1 - mu**2 - tf.exp(log_var))
            kl_div = tf.reduce_sum(kl_div) / x.shape[0]

            loss = rec_loss + 1. * kl_div

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))


        if step % 100 == 0:
            print(epoch, step, 'kl div:', float(kl_div), 'rec loss:', float(rec_loss))


    # evaluation
    z = tf.random.normal((batchsz, z_dim))
    logits = model.decoder(z)
    x_hat = tf.sigmoid(logits)
    x_hat = tf.reshape(x_hat, [-1, 28, 28]).numpy() *255.
    x_hat = x_hat.astype(np.uint8)
    save_images(x_hat, 'vae_images/sampled_epoch%d.png'%epoch)

    x = next(iter(test_db))
    x = tf.reshape(x, [-1, 784])
    x_hat_logits, _, _ = model(x)
    x_hat = tf.sigmoid(x_hat_logits)
    x_hat = tf.reshape(x_hat, [-1, 28, 28]).numpy() *255.
    x_hat = x_hat.astype(np.uint8)
    save_images(x_hat, 'vae_images/rec_epoch%d.png'%epoch)

0 0 kl div: 2.151616334915161 rec loss: 545.9842529296875
0 100 kl div: 17.32524299621582 rec loss: 287.68841552734375
1 0 kl div: 17.233699798583984 rec loss: 273.8408508300781
1 100 kl div: 15.523046493530273 rec loss: 260.31884765625
2 0 kl div: 16.024169921875 rec loss: 257.2443542480469
2 100 kl div: 15.48560905456543 rec loss: 250.13624572753906
3 0 kl div: 15.154146194458008 rec loss: 243.53298950195312
3 100 kl div: 15.049654006958008 rec loss: 245.84500122070312
4 0 kl div: 14.9567232131958 rec loss: 244.17489624023438
4 100 kl div: 14.775262832641602 rec loss: 245.951904296875
5 0 kl div: 14.598281860351562 rec loss: 241.92825317382812
5 100 kl div: 14.521360397338867 rec loss: 242.98626708984375
6 0 kl div: 15.11983871459961 rec loss: 238.15992736816406
6 100 kl div: 14.396514892578125 rec loss: 242.870849609375
7 0 kl div: 15.154908180236816 rec loss: 235.40689086914062
7 100 kl div: 15.249704360961914 rec loss: 234.8616943359375
8 0 kl div: 14.695381164550781 rec loss: 234

67 100 kl div: 14.884611129760742 rec loss: 232.39288330078125
68 0 kl div: 15.020929336547852 rec loss: 228.650634765625
68 100 kl div: 14.973621368408203 rec loss: 225.39700317382812
69 0 kl div: 14.831850051879883 rec loss: 219.13055419921875
69 100 kl div: 15.08685302734375 rec loss: 233.18572998046875
70 0 kl div: 15.193010330200195 rec loss: 224.3316650390625
70 100 kl div: 15.271081924438477 rec loss: 228.65516662597656
71 0 kl div: 15.246682167053223 rec loss: 230.28662109375
71 100 kl div: 15.541927337646484 rec loss: 230.0749053955078
72 0 kl div: 15.090825080871582 rec loss: 221.94976806640625
72 100 kl div: 15.256498336791992 rec loss: 227.07363891601562
73 0 kl div: 14.994352340698242 rec loss: 223.62278747558594
73 100 kl div: 15.325594902038574 rec loss: 226.25216674804688
74 0 kl div: 15.253377914428711 rec loss: 226.8294219970703
74 100 kl div: 15.128423690795898 rec loss: 230.72140502929688
75 0 kl div: 15.174392700195312 rec loss: 221.19808959960938
75 100 kl div: 15

134 0 kl div: 15.521451950073242 rec loss: 224.4097900390625
134 100 kl div: 15.261245727539062 rec loss: 220.16552734375
135 0 kl div: 14.954909324645996 rec loss: 221.05856323242188
135 100 kl div: 15.007464408874512 rec loss: 224.30621337890625
136 0 kl div: 14.907127380371094 rec loss: 228.93402099609375
136 100 kl div: 15.249629974365234 rec loss: 228.92990112304688
137 0 kl div: 14.890340805053711 rec loss: 224.28472900390625
137 100 kl div: 14.892776489257812 rec loss: 225.22732543945312
138 0 kl div: 14.900287628173828 rec loss: 217.8429718017578
138 100 kl div: 15.601009368896484 rec loss: 230.13377380371094
139 0 kl div: 15.43714714050293 rec loss: 221.20928955078125
139 100 kl div: 15.094841003417969 rec loss: 220.53652954101562
140 0 kl div: 14.735057830810547 rec loss: 230.44552612304688
140 100 kl div: 15.004205703735352 rec loss: 227.0232696533203
141 0 kl div: 14.751779556274414 rec loss: 220.90760803222656
141 100 kl div: 15.072603225708008 rec loss: 223.50743103027344

200 0 kl div: 14.781783103942871 rec loss: 219.8212432861328
200 100 kl div: 14.82581615447998 rec loss: 229.64718627929688
201 0 kl div: 15.608495712280273 rec loss: 218.14956665039062
201 100 kl div: 14.868600845336914 rec loss: 232.21546936035156
202 0 kl div: 15.126910209655762 rec loss: 220.83560180664062
202 100 kl div: 15.04362678527832 rec loss: 227.9005889892578
203 0 kl div: 15.052898406982422 rec loss: 218.3063507080078
203 100 kl div: 14.561360359191895 rec loss: 230.72520446777344
204 0 kl div: 14.464433670043945 rec loss: 221.9228515625
204 100 kl div: 15.118017196655273 rec loss: 222.96603393554688
205 0 kl div: 15.122743606567383 rec loss: 219.6361541748047
205 100 kl div: 14.902972221374512 rec loss: 222.015380859375
206 0 kl div: 15.58582592010498 rec loss: 222.49269104003906
206 100 kl div: 15.052011489868164 rec loss: 230.8338165283203
207 0 kl div: 15.176218032836914 rec loss: 220.13214111328125
207 100 kl div: 14.986434936523438 rec loss: 225.3422393798828
208 0 k

266 0 kl div: 15.539977073669434 rec loss: 220.25289916992188
266 100 kl div: 15.039884567260742 rec loss: 224.13265991210938
267 0 kl div: 15.092823028564453 rec loss: 215.42889404296875
267 100 kl div: 14.887808799743652 rec loss: 230.71185302734375
268 0 kl div: 14.747845649719238 rec loss: 223.07479858398438
268 100 kl div: 14.852300643920898 rec loss: 225.35586547851562
269 0 kl div: 14.859052658081055 rec loss: 223.93954467773438
269 100 kl div: 15.219866752624512 rec loss: 227.1107635498047
270 0 kl div: 15.141922950744629 rec loss: 226.65548706054688
270 100 kl div: 14.759958267211914 rec loss: 230.0352020263672
271 0 kl div: 15.054861068725586 rec loss: 219.96568298339844
271 100 kl div: 15.11562728881836 rec loss: 227.3756561279297
272 0 kl div: 14.479601860046387 rec loss: 222.67735290527344
272 100 kl div: 14.782814979553223 rec loss: 228.99591064453125
273 0 kl div: 14.748405456542969 rec loss: 218.44985961914062
273 100 kl div: 15.194051742553711 rec loss: 220.67018127441

...

728 0 kl div: 14.817825317382812 rec loss: 216.72979736328125
728 100 kl div: 14.711542129516602 rec loss: 223.5373992919922
729 0 kl div: 15.352850914001465 rec loss: 225.6110382080078
729 100 kl div: 14.643177032470703 rec loss: 223.39309692382812
730 0 kl div: 14.662989616394043 rec loss: 221.37046813964844
730 100 kl div: 15.15524673461914 rec loss: 221.18710327148438
731 0 kl div: 14.690326690673828 rec loss: 227.47364807128906
731 100 kl div: 15.285487174987793 rec loss: 225.73165893554688
732 0 kl div: 14.953692436218262 rec loss: 221.6600799560547
732 100 kl div: 14.809869766235352 rec loss: 222.551513671875
733 0 kl div: 15.104341506958008 rec loss: 220.39585876464844
733 100 kl div: 15.057523727416992 rec loss: 225.8860626220703
734 0 kl div: 14.459866523742676 rec loss: 223.8697967529297
734 100 kl div: 14.885562896728516 rec loss: 220.19772338867188
735 0 kl div: 14.83228588104248 rec loss: 223.45919799804688
735 100 kl div: 14.480192184448242 rec loss: 221.18548583984375
7

794 0 kl div: 14.768186569213867 rec loss: 223.22763061523438
794 100 kl div: 14.600208282470703 rec loss: 221.82568359375
795 0 kl div: 14.908792495727539 rec loss: 221.35479736328125
795 100 kl div: 14.863202095031738 rec loss: 223.41183471679688
796 0 kl div: 14.686453819274902 rec loss: 218.87220764160156
796 100 kl div: 15.12611198425293 rec loss: 225.1491241455078
797 0 kl div: 14.863718032836914 rec loss: 222.39503479003906
797 100 kl div: 14.606698036193848 rec loss: 228.34645080566406
798 0 kl div: 14.764204978942871 rec loss: 226.28030395507812
798 100 kl div: 15.002748489379883 rec loss: 225.17860412597656
799 0 kl div: 14.702770233154297 rec loss: 219.59324645996094
799 100 kl div: 14.731011390686035 rec loss: 223.110107421875
800 0 kl div: 15.142252922058105 rec loss: 220.41586303710938
800 100 kl div: 14.688116073608398 rec loss: 221.46888732910156
801 0 kl div: 14.812932968139648 rec loss: 221.85008239746094
801 100 kl div: 14.940584182739258 rec loss: 226.24053955078125

860 0 kl div: 15.064697265625 rec loss: 226.1486358642578
860 100 kl div: 14.497419357299805 rec loss: 227.2784423828125
861 0 kl div: 14.849228858947754 rec loss: 220.9403533935547
861 100 kl div: 14.850749969482422 rec loss: 221.12619018554688
862 0 kl div: 14.703903198242188 rec loss: 222.79281616210938
862 100 kl div: 14.730478286743164 rec loss: 228.2082061767578
863 0 kl div: 14.867209434509277 rec loss: 223.12103271484375
863 100 kl div: 14.967002868652344 rec loss: 225.2834930419922
864 0 kl div: 14.845284461975098 rec loss: 224.98011779785156
864 100 kl div: 14.928539276123047 rec loss: 224.61668395996094
865 0 kl div: 14.719265937805176 rec loss: 225.45870971679688
865 100 kl div: 14.635290145874023 rec loss: 229.5419464111328
866 0 kl div: 15.00905990600586 rec loss: 224.18411254882812
866 100 kl div: 14.818405151367188 rec loss: 226.80398559570312
867 0 kl div: 14.968788146972656 rec loss: 224.15460205078125
867 100 kl div: 14.833450317382812 rec loss: 222.69854736328125
86

926 0 kl div: 14.617161750793457 rec loss: 224.44471740722656
926 100 kl div: 14.437433242797852 rec loss: 226.63092041015625
927 0 kl div: 14.77416706085205 rec loss: 225.02268981933594
927 100 kl div: 14.944284439086914 rec loss: 224.64697265625
928 0 kl div: 15.26429271697998 rec loss: 221.14671325683594
928 100 kl div: 14.905607223510742 rec loss: 221.28648376464844
929 0 kl div: 14.68588924407959 rec loss: 216.99142456054688
929 100 kl div: 14.613636016845703 rec loss: 220.8770294189453
930 0 kl div: 14.891971588134766 rec loss: 219.3250274658203
930 100 kl div: 14.773096084594727 rec loss: 225.3357696533203
931 0 kl div: 15.043230056762695 rec loss: 228.14146423339844
931 100 kl div: 14.956452369689941 rec loss: 221.12298583984375
932 0 kl div: 14.66917610168457 rec loss: 220.86610412597656
932 100 kl div: 14.882476806640625 rec loss: 224.63038635253906
933 0 kl div: 15.10919189453125 rec loss: 222.8142547607422
933 100 kl div: 14.720081329345703 rec loss: 222.31764221191406
934 

992 0 kl div: 14.462810516357422 rec loss: 222.26409912109375
992 100 kl div: 14.688316345214844 rec loss: 226.2683563232422
993 0 kl div: 14.940780639648438 rec loss: 222.2132568359375
993 100 kl div: 14.93855094909668 rec loss: 225.11595153808594
994 0 kl div: 14.95480728149414 rec loss: 222.5709228515625
994 100 kl div: 14.773487091064453 rec loss: 221.26516723632812
995 0 kl div: 14.849409103393555 rec loss: 221.64260864257812
995 100 kl div: 14.856775283813477 rec loss: 221.89337158203125
996 0 kl div: 14.903229713439941 rec loss: 220.27020263671875
996 100 kl div: 14.967842102050781 rec loss: 226.49098205566406
997 0 kl div: 15.289575576782227 rec loss: 224.71533203125
997 100 kl div: 14.782672882080078 rec loss: 218.16537475585938
998 0 kl div: 14.768160820007324 rec loss: 218.03338623046875
998 100 kl div: 14.74802303314209 rec loss: 219.87034606933594
999 0 kl div: 14.926408767700195 rec loss: 222.33987426757812
999 100 kl div: 14.74444580078125 rec loss: 226.5708465576172
